In [1]:

#!python3 -m pip install pytorch_lightning
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model import LSTM_Model
from pytorch_lightning import seed_everything, LightningModule, Trainer
from torch import save
from pytorch_lightning.callbacks import EarlyStopping

torch.manual_seed(1)

Dataset version: 2.0


In [ ]:
seed_everything(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
#device = 'cpu'
print(f'Using {device} device')


model = LSTM_Model('data/bcm/').to(device)

trainer = Trainer(max_epochs=20, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False,enable_checkpointing=False, accelerator="gpu", devices = 1)
trainer.tune(model)

trainer.fit(model)
#save(model.state_dict(), '/trained_model')

Global seed set to 42


Using cuda device
data/bcm//3.npy
data/bcm//4.npy
data/bcm//2.npy
data/bcm//1.npy
data/bcm//0.npy


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type             | Params
--------------------------------------------------
0 | lstm         | LSTM             | 42.0 K
1 | fc           | Linear           | 645   
2 | flatten      | Flatten          | 0     
3 | output       | Sigmoid          | 0     
4 | sm           | Softmax          | 0     
5 | loss         | CrossEntropyLoss | 0     
6 | accuracy     | Accuracy         | 0     
7 | val_accuracy | Accuracy         | 0     
--------------------------------------------------
42.6 K    Trainable params
0         Non-trainable params
42.6 K    Total params
0.171     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

{'progress_bar': {'val_loss': 1.6113076210021973}, 'log': {'val_loss': 1.6113076210021973}, 'val_loss': 1.6113076210021973}
Accuracy: 0.125


...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



{'progress_bar': {'val_loss': 1.4115393161773682}, 'log': {'val_loss': 1.4115393161773682}, 'val_loss': 1.4115393161773682}
Accuracy: 0.9967710971832275
Accuracy: 0.8803249001502991


Validation: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



{'progress_bar': {'val_loss': 1.4116653203964233}, 'log': {'val_loss': 1.4116653203964233}, 'val_loss': 1.4116653203964233}
Accuracy: 0.9953681826591492
Accuracy: 0.9958915114402771
